In [ ]:
from bs4 import BeautifulSoup 
import requests
import pandas as pd

In [ ]:
url = "https://boston.craigslist.org/search/npo"

In [ ]:
#create a disctionary 

d = {'key' : 'value'}
print(d)

In [ ]:
#update the dictionary 

d ['new key'] = 'new value'
print(d)

In [ ]:
npo_jobs = {}
job_no = 0

while True:
    
    response = requests.get(url)
    data = response.text
    soup = BeautifulSoup(data, 'html.parser')
    jobs = soup.find_all('p',{'class': 'result-info'})
    
    for job in jobs:
        
        title = job.find('a', {'class':'result-title'}).text
        location_tag = job.find('span', {'class':'result-hood'})
        location = location_tag.text[1:-1] if location_tag else "N/A"
        data = job.find('time', {'class':'result-date'}).text
        link = job.find('a',{'class':'result-title'}).get('href')
        
        job_response = requests.get(link)
        job_data = job_response.text
        job_soup = BeautifulSoup(job_data, 'html.parser')
        job_description = job_soup.find('section',{'id':'postingbody'}).text
        job_attributes_tag = job_soup.find('p',{'class':'attrgroup'})
        job_attributes = job_attributes_tag.text if job_attributes_tag else "N/A"
        
        job_no+=1
        npo_jobs[job_no] = [title, location, data, link, job_attributes, job_description]
        
       # print('Job Title:', title, '\nLocation:', location, '\nDate:', date,'\nlink:', link, '\n', Job Attributes:, '\nJob Description:', job_description
        
    url_tag = soup.find('a',{'title':'next page'})
    if url_tag.get('href'):
        url='https://boston.craigslist.org' + url_tag.get('href')
        print(url)
    else:
        break
    
    print("Total Jobs:", job_no)
    npo_jobs_df = pd.DataFrame.from_dict(npo_jobs, orient = 'index', columns = ['Job Title','Location','Date','Link','Job Attributes','Job Desciption'])
    
        
        
        

In [126]:
!ls

BostonJobs.ipynb annajoen.ipynb   exoneration.csv


In [128]:
npo_jobs_df.head()

,Job Title,Location,Date,Link,Job Attributes,Job Desciption
1,Private Care Manager,"(Waltham, MA",Nov 29,https://boston.craigslist.org/gbs/npo/d/waltha...,\ncompensation: Competitive salary\n\nemployme...,\n\nQR Code Link to This Post\n\n\nSpringwell ...
2,Community Health Worker,"(Quincy, MA",Nov 29,https://boston.craigslist.org/sob/npo/d/quincy...,\ncompensation: Competitive salary\n\nemployme...,\n\nQR Code Link to This Post\n\n\nSpringwell ...
3,Part-Time Floating Nutrition Assistant,"(Waltham, MA",Nov 29,https://boston.craigslist.org/gbs/npo/d/waltha...,\ncompensation: $15.00/hour\n\nemployment type...,\n\nQR Code Link to This Post\n\n\nSpringwell ...
4,"Part-Time Dining Site Assistant, $15/hour","(Waltham, MA",Nov 29,https://boston.craigslist.org/gbs/npo/d/waltha...,\ncompensation: $15.00/hour\n\nemployment type...,\n\nQR Code Link to This Post\n\n\nCommunity N...
5,"Part-Time Home Delivered Meals Drivers, $15/hour","(Waltham, MA",Nov 29,https://boston.craigslist.org/gbs/npo/d/waltha...,\ncompensation: $15.00/hour\n\nemployment type...,\n\nQR Code Link to This Post\n\n\nLooking to ...


In [129]:
npo_jobs_df.to_csv('npo_jobs.csv')